In [1]:
import torch
import tensorflow as tf
import pprint
from datasets import load_dataset
import openai
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from IPython.display import display, HTML
import time

2024-10-18 15:54:13.343085: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 15:54:13.977683: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-18 15:54:15.577725: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 15:54:15.766619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 15:54:15.838050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [2]:
print(torch.cuda.is_available())  # Should return True if CUDA is available

True


In [3]:
# Reading API Key from file
API_KEY = open("open_ai_API.txt", "r").read().strip()
openai.api_key = API_KEY

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
data = load_dataset("generaleoley/manim-codegen", split='train')
# print(data)

In [6]:
from utils import llm_tools, tools_local

lt = llm_tools()
tl = tools_local()

In [7]:
# start_time = time.time()
# #Choosing a specific code from the dataset by giving value of 'reference', change value for each code 

# sorted_set = []

# for j in range(3):
    
#     reference = j
#     code_string = data[reference]['answer']
#     formatted_code_string = code_string.replace('\\n', '\n')
#     # print(formatted_code_string)

#     # Prepare the messages for the Chat API which will be used later for cosine similarity
#     messages = [
#         # {"role": "system", "content": "You are a helpful assistant that performs semantic analysis."},
#         {"role": "user", "content": f"Explain the purpose of this code wihtout getting into technical details, \n\nParagraph 1:\n{formatted_code_string}"},

#         {"role": "user", "content": f"In detail, explain what is happening with the visuals,  \n\nParagraph 1:\n{formatted_code_string} "}
#     ]

#     # Make the API call
#     response = openai.chat.completions.create(
#         model="gpt-4o-mini",  # Or another model as appropriate
#         messages=messages,
#         max_tokens=700,  # Adjust according to your needs
#         temperature=0.01  # Adjust for more creativity or precision
#     )
#     open_gen = response.choices[0].message.content
#     # printer(open_gen)
    
    
#     model1 = SentenceTransformer('all-MiniLM-L6-v2').to(device)


# ## Convert to function ####

# ## Ask gpt if it fits ####

# ### Remove correct pairs #####

#     top_similarities = lt.similarity_ranking(data,model1, open_gen, 10)
    
#     all_scores = [item['score'] for item in top_similarities]
#     all_indices = [item['index'] for item in top_similarities]
#     all_queries = [item['query'] for item in top_similarities]
        
#     messages2 = lt.create_message_for_comparison(code_string, all_queries)

#     # Make the API call to GPT-4o-mini for comparison
#     response = openai.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=messages2,
#         max_tokens=700,
#         temperature=0.01
#     )

#     # Extract and return the most similar query from the response
#     response_content = response.choices[0].message.content
#     # printer(response_content)
    
#     searched_index = tl.searching(response_content, reference, data)
#     print(searched_index)

#     sorted_set.append(searched_index)


# end_time = time.time()
# print(f"Exectution time :",end_time - start_time)

In [8]:
# sorted_set

In [9]:
start_time = time.time()

# Initialize a list to store results of all runs
all_runs_searched_indices = []
all_runs_top_similarities = []  # To store the top similarities for each run

# Run the loop 5 times
for run in range(4):
    
    sorted_set = []  # This will store the result of each run
    run_top_similarities = []  # To store top similarities for the current run

    for i in range(3):  # Loop through each code snippet in the dataset
        
        reference = i
        code_string = data[reference]['answer']
        formatted_code_string = code_string.replace('\\n', '\n')

        # Prepare the messages for the Chat API
        messages = [
            {"role": "user", "content": f"Explain the purpose of this code without getting into technical details, \n\nParagraph 1:\n{formatted_code_string}"},
            {"role": "user", "content": f"In detail, explain what is happening with the visuals,  \n\nParagraph 1:\n{formatted_code_string} "}
        ]

        # Make the API call for explanation
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=700,
            temperature=0.01
        )

        open_gen = response.choices[0].message.content

        model1 = SentenceTransformer('all-MiniLM-L6-v2').to(device) 
        
        top_similarities = lt.similarity_ranking(data, model1, open_gen, 5)

        # Store top scores, indices, and queries
        all_scores = [item['score'] for item in top_similarities]
        all_indices = [item['index'] for item in top_similarities]
        all_queries = [item['query'] for item in top_similarities]

        # Save top indices and scores for the current run
        run_top_similarities.append({
            'iteration': i,
            'indices': all_indices,
            'scores': all_scores
        })


        # Prepare messages for comparison
        messages2 = lt.create_message_for_comparison(code_string, all_queries)

        # Make the API call for comparison
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages2,
            max_tokens=700,
            temperature=0.01
        )


        # Extract the most similar query from the response
        response_content = response.choices[0].message.content


        # Perform searching based on response content and reference
        searched_index = tl.searching(response_content, reference, data)
        print(f"Run {run}, Iteration {i}, Searched Index: {searched_index}")


        # Append the result to sorted_set
        sorted_set.append(searched_index)


    # Store the result of the current run
    all_runs_searched_indices.append(sorted_set)
    all_runs_top_similarities.append(run_top_similarities)  # Append the run's top similarities
    print(f"Completed Run {run}")


end_time = time.time()


# Output execution time
print(f"Execution time: {end_time - start_time}")


# Accessing the searched indices for each run
for run_idx, run_results in enumerate(all_runs_searched_indices):
    print(f"Results from Run {run_idx + 1}: {run_results}")


/scratch/h/Hassan.Mo/llm_env/lib64/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Run 0, Iteration 0, Searched Index: [(0, 771)]
Run 0, Iteration 1, Searched Index: [(1, 990)]
Run 0, Iteration 2, Searched Index: []
Completed Run 0
Run 1, Iteration 0, Searched Index: [(0, 771)]
Run 1, Iteration 1, Searched Index: [(1, 990)]
Run 1, Iteration 2, Searched Index: [(2, 804)]
Completed Run 1
Run 2, Iteration 0, Searched Index: [(0, 771)]
Run 2, Iteration 1, Searched Index: [(1, 990)]
Run 2, Iteration 2, Searched Index: [(2, 599)]
Completed Run 2
Run 3, Iteration 0, Searched Index: [(0, 506)]
Run 3, Iteration 1, Searched Index: [(1, 990)]
Run 3, Iteration 2, Searched Index: [(2, 804)]
Completed Run 3
Execution time: 332.8417229652405
Results from Run 1: [[(0, 771)], [(1, 990)], []]
Results from Run 2: [[(0, 771)], [(1, 990)], [(2, 804)]]
Results from Run 3: [[(0, 771)], [(1, 990)], [(2, 599)]]
Results from Run 4: [[(0, 506)], [(1, 990)], [(2, 804)]]


In [10]:
# Accessing the top similarities for each run
for run_idx, run_top in enumerate(all_runs_top_similarities):
    print(f"Top Similarities from Run {run_idx + 1}:")
    for item in run_top:
        print(f"Iteration {item['iteration']} - Indices: {item['indices']}, Scores: {item['scores']}")

Top Similarities from Run 1:
Iteration 0 - Indices: [771, 802, 490, 620, 581], Scores: [0.8299391269683838, 0.8209885954856873, 0.8169922232627869, 0.8050893545150757, 0.7997460961341858]
Iteration 1 - Indices: [990, 986, 22, 25, 987], Scores: [0.6782309412956238, 0.6243355870246887, 0.6078952550888062, 0.6071741580963135, 0.606212854385376]
Iteration 2 - Indices: [751, 745, 599, 804, 540], Scores: [0.7789125442504883, 0.7719681859016418, 0.7715654373168945, 0.7652341723442078, 0.7648677825927734]
Top Similarities from Run 2:
Iteration 0 - Indices: [512, 490, 581, 771, 631], Scores: [0.8192092180252075, 0.8120673894882202, 0.8066164255142212, 0.8017783164978027, 0.8006011843681335]
Iteration 1 - Indices: [990, 986, 22, 25, 954], Scores: [0.6578432321548462, 0.614272952079773, 0.6069788932800293, 0.6032810807228088, 0.5942022800445557]
Iteration 2 - Indices: [599, 804, 430, 751, 520], Scores: [0.7449238300323486, 0.7412062883377075, 0.7324252128601074, 0.7303513288497925, 0.726993024349